<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# EWHA bagging (3)

In [1]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()

file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [2]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
#query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

In [3]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

## 1. build DB

In [4]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os
import numpy as np


UPSTAGE_API_KEY = api_key

# .npy 파일 로드 (타입==넘파이)
ewhaDB = np.load(data_path+f'embedding/full_ewha500.npy')

ewhaDB_embed = np.load(data_path+f'embedding/full_ewha500_embed.npy')
ewhaDB_embed = ewhaDB_embed.tolist()

## 3. test set 갖고오기

In [5]:
# read samples.csv file
import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [6]:
prompts, answers = read_data(os.path.join(data_path, 'test_own_ewha.csv'))
testdata = pd.read_csv(data_path+'test_own_ewha.csv')

In [7]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers', 'top1', 'top2', 'top3', 'top1pred', 'top2pred', 'top3pred', 'predict' ])

for index, row in testdata.iterrows():
    #if index % 100 != 0 : continue # 일단 실험할 땐 100개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]
    try : 
        embedded_query = passage_embeddings.embed_query(question) # 질문만 받아와서 embedding 하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a, 'top1': None, 'top2': None, 'top3': None,'top1': None, 'top2': None, 'top3': None, 'predict': None, }

    except :
        print(f'pass: {index}')
        continue 


In [8]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3,top1pred,top2pred,top3pred,predict
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),None,None,None,NaN,NaN,NaN,None
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),None,None,None,NaN,NaN,NaN,None
2,2,"[0.007404327392578125, -0.0216522216796875, 0....",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),None,None,None,NaN,NaN,NaN,None
3,3,"[-0.002162933349609375, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),None,None,None,NaN,NaN,NaN,None
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),None,None,None,NaN,NaN,NaN,None
5,5,"[0.0016117095947265625, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),None,None,None,NaN,NaN,NaN,None
6,6,"[-0.018157958984375, -0.0157318115234375, -0.0...",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),None,None,None,NaN,NaN,NaN,None
7,7,"[0.0029163360595703125, -0.018707275390625, -0...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),None,None,None,NaN,NaN,NaN,None
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),None,None,None,NaN,NaN,NaN,None
9,9,"[-0.002269744873046875, 0.0017795562744140625,...",학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A...,(B),None,None,None,NaN,NaN,NaN,None


## 4. Prompt engineering

In [9]:
import numpy as np

for idx, row in nowtest.iterrows() : # 질문 받아오기 

    embed_ques= row.embed_ques

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embed_ques) @ np.array(ewhaDB_embed).T).argsort()[::-1]

    nowtest.loc[idx, 'top1'] = ewhaDB[sorted_idx[0]]
    nowtest.loc[idx, 'top2'] = ewhaDB[sorted_idx[1]]
    nowtest.loc[idx, 'top3'] = ewhaDB[sorted_idx[2]]


In [10]:
try : del [[bagging_pred]]
except : pass
bagging_pred = pd.DataFrame(columns=['questionNum', 'answer', 'top1pred', 'top2pred', 'top3pred', 'predict'])

In [11]:
############# first PREDICTION ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

 
llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    You are an assistant with expertise in Ewha University policies and history. Use the provided context to answer accurately.
    Let's think step by step.

    Context: {context1}

    Question: {question}

    Final Answer:
        
    '''

)
ko_chain1 = prompt_template | llm

responses = []
for idx, row in nowtest.iterrows() :
    #if idx == 50 : break
        
    response = ko_chain1.invoke({"question": row.prompts, "context1": row.top1}) # 선지 전까지 받아오기
    responses.append(response.content)
    nowtest.loc[idx, 'top1pred'] = response.content

    generated_answer = extract_answer(response.content)
    bagging_pred.loc[len(bagging_pred)] = {'questionNum': row.question, 'answer': row.answers, 'top1pred': generated_answer, 'top2pred':None, 'top3pred':None, 'predict':None}


/tmp/ipykernel_429506/2309160220.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '(D) 3학점' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, 'top1pred'] = response.content


In [12]:
############# second PREDICTION ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

 
llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    You are an assistant with expertise in Ewha University policies and history. Use the provided context to answer accurately.
    Let's think step by step.

    Context: {context1} {context2}

    Question: {question}

    Final Answer:
        
    '''

)
ko_chain2 = prompt_template | llm

responses = []
for idx, row in nowtest.iterrows() :
    #if idx == 50 : break
        
    response = ko_chain2.invoke({"question": row.prompts, "context1": row.top1, "context2":row.top2}) # 선지 전까지 받아오기
    responses.append(response.content)
    nowtest.loc[idx, 'top2pred'] = response.content

    generated_answer = extract_answer(response.content)
    bagging_pred.loc[idx, 'top2pred'] = generated_answer


/tmp/ipykernel_429506/506231592.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '(D) 3학점

이유:
제공된 문맥에서 이화여자대학교 학칙 제35조의3에 따르면, 재학 중 국내외의 다른 학교에서 학점을 취득한 경우, 졸업에 필요한 학점의 2분의 1의 범위 안에서 이를 본교에서 취득한 것으로 인정할 수 있습니다. 이 경우 인정받을 수 있는 최대 학점은 12학점입니다. 따라서 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우에도 이 범위에 따라 3학점을 인정받을 수 있습니다.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, 'top2pred'] = response.content


In [13]:
############# third PREDICTION ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

 
llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    You are an assistant with expertise in Ewha University policies and history. Use the provided context to answer accurately.
    Let's think step by step.

    Context: {context1} {context2}

    Question: {question}

    Final Answer:
        
    '''

)
ko_chain3 = prompt_template | llm

responses = []
for idx, row in nowtest.iterrows() :
    #if idx == 50 : break
        
    response = ko_chain3.invoke({"question": row.prompts, "context1": row.top1, "context2":row.top2, "context3":row.top3}) # 선지 전까지 받아오기
    responses.append(response.content)
    nowtest.loc[idx, 'top3pred'] = response.content

    generated_answer = extract_answer(response.content)
    bagging_pred.loc[idx, 'top3pred'] = generated_answer


/tmp/ipykernel_429506/3633945168.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '(C) 2학점

해설: 주어진 문맥에서 "제35조의3(학점의 인정) ① 다음 각 호의 경우에 취득한 학점은 총장의 승인을 얻어 졸업에 필요한 학점의 2분의 1의 범위 안에서 이를 본교에서 취득한 것으로 본다. (개정 2012.12.31.)"라는 내용이 있습니다. 이는 영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우, 졸업에 필요한 학점의 2분의 1의 범위 안에서 이를 본교에서 취득한 것으로 인정받을 수 있다는 것을 의미합니다.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  nowtest.loc[idx, 'top3pred'] = response.content


In [14]:
bagging_pred

,questionNum,answer,top1pred,top2pred,top3pred,predict
0,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),D,D,C,None
1,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,(E),E,E,E,None
2,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,(B),B,B,B,None
3,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,(B),B,B,B,None
4,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),A,D,D,None
5,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,(D),D,D,D,None
6,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,(C),C,C,C,None
7,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),C,A,A,None
8,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),D,D,D,None
9,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,(B),B,B,B,None


In [15]:
from collections import Counter

final_pred = []
for idx, row in bagging_pred.iterrows() :

    pred = []
    pred.append(row.top1pred)
    pred.append(row.top2pred)
    pred.append(row.top3pred)

    counts = Counter(pred)

    prediction = counts.most_common(1)[0][0]
    if prediction == None :
        try : prediction = counts.most_common(2)[1][0]
        except : pass
    final_pred.append(prediction)
    bagging_pred.loc[idx, 'predict'] = prediction

final_pred

['D',
 'E',
 'B',
 'B',
 'D',
 'D',
 'C',
 'A',
 'D',
 'B',
 'D',
 'C',
 'B',
 'D',
 'A',
 'C',
 'E',
 'B',
 'C',
 'B',
 'C',
 'D',
 'B',
 'E',
 'B',
 'B',
 'E',
 'C',
 'D',
 'A',
 'E',
 'C',
 'A',
 'B',
 'C',
 'C',
 'B',
 'A',
 'A',
 'I',
 'F',
 'A']

In [16]:
######### 정답 1차 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, final_pred)):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong.append(idx+1)
        bagging_pred.loc[idx, 'iswrong'] = 'X'
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx+1)
        bagging_pred.loc[idx, 'iswrong'] = 'X'
        
acc = cnt/len(answers)*100
print(f"acc: {acc}%")
print()
print("wrong:", wrong)
bagging_pred.loc[len(bagging_pred), 'predict'] = acc

----------
D
idx: 0 | generated answer: D, answer: (D)
----------
E
idx: 1 | generated answer: E, answer: (E)
----------
B
idx: 2 | generated answer: B, answer: (B)
----------
B
idx: 3 | generated answer: B, answer: (B)
----------
D
idx: 4 | generated answer: D, answer: (C)
----------
D
idx: 5 | generated answer: D, answer:  (D)
----------
C
idx: 6 | generated answer: C, answer:  (C)
----------
A
idx: 7 | generated answer: A, answer:  (D)
----------
D
idx: 8 | generated answer: D, answer:  (C)
----------
B
idx: 9 | generated answer: B, answer:  (B)
----------
D
idx: 10 | generated answer: D, answer:  (B)
----------
C
idx: 11 | generated answer: C, answer:  (C)
----------
B
idx: 12 | generated answer: B, answer:  (B)
----------
D
idx: 13 | generated answer: D, answer:  (E)
----------
A
idx: 14 | generated answer: A, answer:  (A)
----------
C
idx: 15 | generated answer: C, answer:  (C)
----------
E
idx: 16 | generated answer: E, answer:  (E)
----------
B
idx: 17 | generated answer: B, an

In [17]:
bagging_pred

,questionNum,answer,top1pred,top2pred,top3pred,predict,iswrong
0,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),D,D,C,D,NaN
1,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,(E),E,E,E,E,NaN
2,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,(B),B,B,B,B,NaN
3,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,(B),B,B,B,B,NaN
4,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),A,D,D,D,X
5,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,(D),D,D,D,D,NaN
6,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,(C),C,C,C,C,NaN
7,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),C,A,A,A,X
8,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),D,D,D,D,X
9,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,(B),B,B,B,B,NaN


In [18]:
wdf = bagging_pred[bagging_pred['iswrong']=='X']
wdf.loc[len(bagging_pred), 'predict'] = acc

wdf

/tmp/ipykernel_429506/3037078640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wdf.loc[len(bagging_pred), 'predict'] = acc


,questionNum,answer,top1pred,top2pred,top3pred,predict,iswrong
4,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),A,D,D,D,X
7,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),C,A,A,A,X
8,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),D,D,D,D,X
10,학칙 제39조(성적)에 따라 다음 중 성적이 F로 처리되지 않는 경우는 무엇입니까?\n,(B),D,E,D,D,X
13,학칙 제12조(휴업일)에서 명시된 정기휴업일에 해당하지 않는 날은 무엇입니까?\n,(E),D,D,D,D,X
19,학칙 제45조(취득학점)에 따라 직전 학기 평균성적 3.75 이상을 받은 학생이 한...,(C),B,B,B,B,X
23,학칙 제45조(취득학점)에 따라 다음 중 의과대학 의학과 학생이 한 학기에 취득할 ...,(C),E,B,E,E,X
30,다음 중 2018학년도 인문과학대학 기독교학과와 엘텍공과대학 휴먼기계바이오공학부의 ...,(B),None,E,None,E,X
33,제48조의4와 제48조의5의 규정에 대한 설명으로 옳지 않은 것은 무엇인가?\n,(C),B,B,C,B,X
34,다음은 제47조의 전공 유형에 관한 규정이다. 이에 대한 설명으로 올바르게 매칭된 ...,(D),C,B,D,C,X
